In [ ]:
import os
os.listdir('../input/')

In [ ]:
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
import gc

In [ ]:
pd.set_option('max_columns', 500)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
tr_trx = dd.read_csv('../input/train_transaction.csv', blocksize=25e6)
ts_trx = dd.read_csv('../input/test_transaction.csv', blocksize=25e6)

In [ ]:
tr_id = dd.read_csv('../input/train_identity.csv', blocksize=25e6)
ts_id = dd.read_csv('../input/test_identity.csv', blocksize=25e6)

In [ ]:
# tr_trx.head(npartitions=1)

In [ ]:
# tr_id.head(npartitions=1)

In [ ]:
tr = dd.merge(tr_trx, tr_id, on='TransactionID', how='inner')
ts = dd.merge(ts_trx, ts_id, on='TransactionID', how='inner')

In [ ]:
del(tr_trx)
del(tr_id)
del(ts_id)
del(ts_trx)
gc.collect()

In [ ]:
seed = 44

In [ ]:
tr = tr.sample(frac=1, random_state=seed)

In [ ]:
# tr['isFraud'].value_counts().compute()

In [ ]:
# df_0 = tr[tr['isFraud'] == 0]
# df_1 = tr[tr['isFraud'] == 1]

In [ ]:
# df_0_u = df_0.sample(frac = (20663/590540), random_state=seed)

In [ ]:
# df = dd.concat([df_0_u, df_1], 0)

In [ ]:
# df = df.persist()

In [ ]:
train = tr.persist()

In [ ]:
test = ts.persist()

In [ ]:
del(tr)
del(ts)
gc.collect()

In [ ]:
# df.compute().shape

In [ ]:
train.compute().shape, test.compute().shape

In [ ]:
# df['isFraud'].value_counts().compute()

In [ ]:
train.index = train['TransactionID']
test.index = test['TransactionID']

In [ ]:
train = train.drop(['TransactionID'], 1)
test = test.drop(['TransactionID'], 1)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
target = train['isFraud']
features = train.drop(['isFraud'], 1)

In [ ]:
data = dd.concat([features, test], 0)

In [ ]:
cat_cols = data.select_dtypes('object').columns

In [ ]:
data = dd.get_dummies(data.categorize(), cat_cols, drop_first=True)

In [ ]:
# data.head()

In [ ]:
data = data.drop(['TransactionDT'], 1)

In [ ]:
train.compute().shape, test.compute().shape

In [ ]:
features = data.compute().head(144233)
test_df = data.compute().tail(141907)

In [ ]:
del(data)
del(train)
# del(test)
gc.collect()

In [ ]:
features = features.fillna(features.mean())
test_df = test_df.fillna(test_df.mean())

features = features.fillna(0)
test_df = test_df.fillna(0)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
X = pca.fit_transform(features)

In [ ]:
pca.explained_variance_ratio_

In [ ]:
features.shape, target.compute().shape

In [ ]:
from sklearn.model_selection import train_test_split

x, x_test, y, y_test = train_test_split(X, target.compute(), test_size=0.2, random_state=seed)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=seed)

In [ ]:
from imblearn.over_sampling import SMOTE

smt = SMOTE(random_state=seed)
x_train, y_train = smt.fit_sample(x_train, y_train)

In [ ]:
# from imblearn.under_sampling import NearMiss

# nm = NearMiss(random_state=seed)
# x_train, y_train = nm.fit_sample(x_train, y_train)

In [ ]:
import numpy as np
np.bincount(y_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier


model = RandomForestClassifier(random_state=seed)
model.fit(x_train, y_train)

In [ ]:
y_val_pred = model.predict(x_val)
y_test_pred = model.predict(x_test)

In [ ]:
print(model.score(x_val, y_val))
print(model.score(x_test, y_test))

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

print(sns.heatmap(confusion_matrix(y_test, y_test_pred), annot=True, fmt='g'))
print(f1_score(y_test, y_test_pred))

In [ ]:
np.bincount(y_test)

In [ ]:
test_features = pca.transform(test_df)

In [ ]:
predictions = model.predict(test_features)

In [ ]:
submission = pd.DataFrame({'TransactionID': test.compute().index, 'isFraud': predictions})

In [ ]:
submission['isFraud'].unique()

In [ ]:
submission.to_csv('submission.csv', index=False)